# Sentiment analyse i Python

## Sentiment analyse med `asent` i Python

I denne notebook vises, hvordan man kan udføre sentiment analyse i Python.

Der bruges pakken [`asent`](https://kennethenevoldsen.github.io/asent/installation.html), som virker sammen med `spaCy` til at udføre sentiment analyse.


## Hvad er asent?

`asent` er en regel-baseret sentiment model, der bygger på en dictionary af ord, som er kategoriseret som hhv. positive og negative ord. Ordene er samtidig givet en polaritetsscore for, hvor positive og negative de er.

Modellen virker ved, at den sammenholder ord i et stykke tekst med ordene i dictionary og giver tekststykket en samlet polaritetsscore for, hvor positivt eller negativt det er.


### Lexicon

Da `asent` er en dictionary-model, skal den bruge en oversigt over ord samt deres tilknyttede score (kaldes ofte "lexicon"). De lexicons, som asent bruger, kan findes på pakkens github-repository: [https://github.com/KennethEnevoldsen/asent/tree/main/src/asent/lexicons](https://github.com/KennethEnevoldsen/asent/tree/main/src/asent/lexicons).

*Bemærk:* `asent` genbruger lexicons fra andre sentiment modeller ([afinn](https://github.com/fnielsen/afinn) og [sentida](https://github.com/Guscode/Sentida)).


### asent i kombination med `spaCy` 

`asent` virker i kombination med en `spaCy` sprogmodel/pipeline. På den måde kan modellen tage højde for sætningskonstruktion og negationer (fx "ikke glad") for at udregne mere præcise polaritetsscores.

`spaCy` modeller er altid et *pipeline*:

![pipeline](https://spacy.io/images/pipeline-design.svg)

Det betyder, at når et stykke tekst behandles med `spaCy` (fx `doc = nlp(text)`), så tages tekststykket igennem flere processer (tokenizer, part-of-speech tagger, dependency parser osv.) - ligesom et samlebånd på en fabrik. 

Man kan altid tilføje flere led i et `spaCy` pipeline - altså endnu en ting, som teksten skal igennem på samlebåndet. Det er den måde, som man bruger `asent`, da den skal bruge information tidligere i pipelinet for at udføre sentiment analysen af teksten. 

Resultatet af sentiment analysen for tekststykket tilgås som attributes for doc-objektet (som man ellers tilgår attributes fra et `spaCy` doc-objekt).


## Dictionary modeller vs. trænede modeller (neurale netværk, transformers etc.)

Dictionary modeller (herunder modeller til sentiment analyse) er efterhånden overgået af præ-trænede maskinlæringsmodeller baseret på transformers eller anden neural netværksarkitektur. Fordelene ved en dictionary model er dog, at det altid er fuldstændig gennemskueligt, hvorfor modellen har analyseret teksten, som den har. Præ-trænede modeller er efterhånden mere nøjagtige, men det er meget vanskeligere for os at spørge modellen, hvorfor den har nået frem til et specifikt resultat ("black-box").

## Brug af `asent`

I det følgende vises, hvordan `asent` bruges på dansk på enkelte tekststykker:

In [6]:
!python -m spacy download da_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.1/567.1 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('da_core_news_lg')


In [1]:
# indlæser pakker
import spacy
import asent
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display, HTML

#python -m spacy download da_core_news_lg

# indlæs spacy sprogmodel og pipeline
nlp = spacy.load('da_core_news_lg') # sprogmodel skal være hentet inden, at den kan indlæses

# tilføj sentiment model til pipeline
nlp.add_pipe('asent_da_v1')

`asent` er nu en del af et `spaCy` pipeline (lagret i `nlp`). Vi er nu klar til at analysere et stykke tekst:

In [2]:
text = "Jeg kan lide grøn peber på varm leverpostej"

doc = nlp(text)

In [ ]:
all_sentences = list(doc.sents)
print(all_sentences[0])

Blot at køre teksten gennem pipeline giver ikke noget output, men alle de dele, som er blevet anlayseret og tilføjet teksten på samlebåndet (pipeline), kan nu tilgås i doc-objektet.

`asent` giver ikke blot én samlet score for hele teksten, men analyserer i stedet sætninger:

In [3]:
for sentence in doc.sents:
    print(f"\nTekst: {sentence}")
    print(f"Sentiment: {sentence._.polarity}")


Tekst: Jeg kan lide grøn peber på varm leverpostej
Sentiment: neg=0.0 neu=0.6 pos=0.4 compound=0.4588 span=Jeg kan lide grøn peber på varm leverpostej


`.sents` er en generator, der giver adgang til alle sætninger i teksten; spaCy's tokenizer opdeler automatisk teksten i sætninger baseret på tegnsætning. Loopet itererer derfor gennem én sætning ad gangen.

Fordi `.sents` er en generator kan vi ikke bruge `doc.sents` eller `print(doc.sents)` direkte. Vi skal først konvertere til en liste `list(doc.sents)[0]` (som i koden tidligere). 

`sentence._` giver adgang til **custom attributes** (tilføjet af asent). `._.` indikerer at det er en custom/user extension (ikke built-in spaCy), `.polarity` er sentiment scoren, som asent har beregnet for denne specifikke sætning




`asent` tildeler både en score for hhv. negativ (`neg`), neutralt (`neu`) og positivt (`pos`). Derudover gives et samlet mål for polaritet (`compound`). Alle scores er normaliseret og rangerer fra -1 til 1. En `compound` score over 0 indikerer derfor overvejende positiv sætning, og under 0 en overvejende negativ sætning.

Denne sætning vurderes overvejende positivt (0.4588). 

### Polaritet på token-niveau

Vi kan inspicere resultatet yderligere ved at se, hvordan de enkelte ord/tokens er vurderet:

In [9]:
print("\nOrd-niveau analyse:")
for token in doc:
    if token._.polarity and token._.polarity != 0:
        print(f"{token.text:15} - Polarity: {token._.polarity}, Valence: {token._.valence}")



Ord-niveau analyse:
lide            - Polarity: polarity=1.0 token=lide span=lide, Valence: 1.0
varm            - Polarity: polarity=1.0 token=varm span=varm, Valence: 1.0


Her ses at ordene "lide" og "varm" vurderes positive (polarity > 0).

`polarity` scores er ikke normaliseret, men er omregninger af de oprindelige tildelte værdier for ordene. De oprindelige tildelte værdier omtales deres "valence" og kan ligeledes tilgås:

In [10]:
for token in doc:
    print(token, "\t", token._.valence)

Jeg 	 0.0
kan 	 0.0
lide 	 1.0
grøn 	 0.0
peber 	 0.0
på 	 0.0
varm 	 1.0
leverpostej 	 0.0


Her er der ingen forskel mellem valence og polarity for ordene. Dette fordi at sætningskonstruktionen ikke lægger op til, at ordene skal vægtes anderledes (se andet eksempel længere nede).

### Visualisering af udregning

`asent` indeholder også visualiseringsfunktioner til at inspicere sentiment analysen:

In [11]:
display(HTML(asent.visualize(doc, style="prediction")))

In [12]:
display(HTML(asent.visualize(doc, style="analysis")))

### Negation og tekster med flere udtryk

Da `asent` benytter `spaCy` pipeline, kan den tage højde for sætningskonstruktion. I det nedenstående bruges `asent` på en sætning med negation og flere udtryk:

In [13]:
text = "Jeg kan ikke lide grøn peber på varm leverpostej. Jeg kan bedre lide rød peber."

doc = nlp(text)

In [14]:
display(HTML(asent.visualize(doc, style="prediction")))

Her ses hvordan `asent` behandler ordet "lide" forskelligt alt efter kontekst. Da det negeres i første sætning, tildeles det en negativ polaritet.

In [15]:
display(HTML(asent.visualize(doc, style="analysis")))

In [16]:
display(HTML(asent.visualize(doc[10:], style="analysis")))

In [17]:
for sentence in doc.sents:
    print(sentence._.polarity)

neg=0.148 neu=0.681 pos=0.17 compound=0.067 span=Jeg kan ikke lide grøn peber på varm leverpostej.
neg=0.0 neu=0.5 pos=0.5 compound=0.6124 span=Jeg kan bedre lide rød peber.


# Med pandas dataframe

In [18]:
df = pd.read_csv('/Users/jeppefl/Library/CloudStorage/OneDrive-AalborgUniversitet/01_work/01_undervisning/02_sds1/03_data/spam.csv', encoding='cp1252')

df = df[['v1', 'v2']].copy()
df.columns = ['label', 'message']
df = df.dropna()

In [19]:
print(f"\nAntal beskeder efter rensning: {len(df)}")
print(f"\nFordeling af spam/ham:")
print(df['label'].value_counts())


Antal beskeder efter rensning: 5572

Fordeling af spam/ham:
label
ham     4825
spam     747
Name: count, dtype: int64


In [6]:
#!python -m spacy download en_core_web_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.8.0/en_core_web_lg-3.8.0-py3-none-any.whl (400.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [20]:
# Skal have installeret: python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe('asent_en_v1')  # Engelsk sentiment model

## Vores egen funktion til at undersøge sentiment i hele corpus

In [21]:
def analyser_sentiment(text):
    """Analyserer sentiment for en enkelt besked"""
    try:
        doc = nlp(text)
        
        # Saml sentiment fra alle sætninger
        sentiments = []
        for sent in doc.sents:
            if sent._.polarity:
                sentiments.append(sent._.polarity)
        
        if not sentiments:
            return None
        
        # Beregn gennemsnit
        avg_compound = np.mean([s.compound for s in sentiments])
        avg_pos = np.mean([s.pos for s in sentiments])
        avg_neg = np.mean([s.neg for s in sentiments])
        avg_neu = np.mean([s.neu for s in sentiments])
        
        return {
            'compound': avg_compound,
            'positive': avg_pos,
            'negative': avg_neg,
            'neutral': avg_neu
        }
    except:
        return None

## Brug funktionen på vores spam-tekster

Tager nogle minutter at kører ...

In [10]:
df_sample = df[:500].copy()  # Fjern [:500] for at analysere alle 5572 beskeder

sentiments = []
for idx, row in df_sample.iterrows():
    if idx % 100 == 0:
        print(f"Har behandlet {idx}/{len(df_sample)} beskeder...")
    
    sentiment = analyser_sentiment(row['message'])
    if sentiment:
        sentiments.append(sentiment)
    else:
        sentiments.append({
            'compound': 0,
            'positive': 0,
            'negative': 0,
            'neutral': 1
        })

# Tilføj sentiment til dataframe
df_sample['compound'] = [s['compound'] for s in sentiments]
df_sample['positive'] = [s['positive'] for s in sentiments]
df_sample['negative'] = [s['negative'] for s in sentiments]
df_sample['neutral'] = [s['neutral'] for s in sentiments]

# Kategorisér sentiment
df_sample['sentiment'] = df_sample['compound'].apply(
    lambda x: 'Positiv' if x >= 0.2 else 'Negativ' if x <= -0.2 else 'Neutral'
)

Behandlet 0/500 beskeder...
Behandlet 100/500 beskeder...


## Analyse resultater

### I hele corpus

In [ ]:
print(f"\nGennemsnitlig sentiment: {df_sample['compound'].mean():.3f}")
print(f"Median sentiment: {df_sample['compound'].median():.3f}")
print(f"Standardafvigelse: {df_sample['compound'].std():.3f}")

print("\nSentiment fordeling:")
print(df_sample['sentiment'].value_counts())

### Sammenligning af kategorier/grupper

In [ ]:
print("\nGennemsnitlig sentiment per type:")
sentiment_by_label = df_sample.groupby('label')['compound'].agg(['mean', 'median', 'std', 'count'])
print(sentiment_by_label)

print("\nSentiment fordeling for SPAM:")
print(df_sample[df_sample['label'] == 'spam']['sentiment'].value_counts())

print("\nSentiment fordeling for HAM:")
print(df_sample[df_sample['label'] == 'ham']['sentiment'].value_counts())

### Visualiseringer

In [ ]:
# Histogram for compound scores
axes[0, 0].hist(df_sample[df_sample['label'] == 'spam']['compound'], 
                bins=30, alpha=0.6, label='Spam', color='red', edgecolor='black')
axes[0, 0].hist(df_sample[df_sample['label'] == 'ham']['compound'], 
                bins=30, alpha=0.6, label='Ham', color='green', edgecolor='black')
axes[0, 0].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Sentiment Score')
axes[0, 0].set_ylabel('Antal')
axes[0, 0].set_title('Fordeling af Sentiment Scores')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

In [ ]:
# Box plot sammenligning af grupper
df_sample.boxplot(column='compound', by='label', ax=axes[0, 1])
axes[0, 1].set_title('Sentiment: Spam vs Ham')
axes[0, 1].set_xlabel('Type')
axes[0, 1].set_ylabel('Sentiment Score')
plt.sca(axes[0, 1])
plt.xticks([1, 2], ['Ham', 'Spam'])

In [ ]:
# Sentiment kategorier per gruppe
sentiment_crosstab = pd.crosstab(df_sample['label'], df_sample['sentiment'], normalize='index') * 100
sentiment_crosstab.plot(kind='bar', ax=axes[0, 2], color=['#e74c3c', '#95a5a6', '#2ecc71'])
axes[0, 2].set_title('Sentiment Fordeling (%)')
axes[0, 2].set_xlabel('Type')
axes[0, 2].set_ylabel('Procent')
axes[0, 2].legend(title='Sentiment')
axes[0, 2].grid(axis='y', alpha=0.3)
plt.sca(axes[0, 2])
plt.xticks(rotation=0)

In [ ]:
# Gennemsnitlige sentiment komponenter
komponenter = df_sample.groupby('label')[['positive', 'negative', 'neutral']].mean()
komponenter.plot(kind='bar', ax=axes[1, 0], color=['#2ecc71', '#e74c3c', '#95a5a6'])
axes[1, 0].set_title('Gennemsnitlige Sentiment Komponenter')
axes[1, 0].set_xlabel('Type')
axes[1, 0].set_ylabel('Score')
axes[1, 0].legend(['Positiv', 'Negativ', 'Neutral'])
axes[1, 0].grid(axis='y', alpha=0.3)
plt.sca(axes[1, 0])
plt.xticks(rotation=0)

In [ ]:
# Scatter plot: Positive vs Negative
spam_data = df_sample[df_sample['label'] == 'spam']
ham_data = df_sample[df_sample['label'] == 'ham']
axes[1, 1].scatter(spam_data['positive'], spam_data['negative'], 
                   alpha=0.5, label='Spam', color='red', s=30)
axes[1, 1].scatter(ham_data['positive'], ham_data['negative'], 
                   alpha=0.5, label='Ham', color='green', s=30)
axes[1, 1].set_xlabel('Positive Score')
axes[1, 1].set_ylabel('Negative Score')
axes[1, 1].set_title('Positiv vs Negativ Komponent')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

## Detaljeret analyse af enkelt tekst

In [ ]:
print("\n--- TOP 3 MEST POSITIVE BESKEDER ---")
top_positive = df_sample.nlargest(3, 'compound')
for idx, row in top_positive.iterrows():
    print(f"\nType: {row['label'].upper()} | Score: {row['compound']:.3f}")
    print(f"Besked: {row['message'][:150]}...")

print("\n--- TOP 3 MEST NEGATIVE BESKEDER ---")
top_negative = df_sample.nsmallest(3, 'compound')
for idx, row in top_negative.iterrows():
    print(f"\nType: {row['label'].upper()} | Score: {row['compound']:.3f}")
    print(f"Besked: {row['message'][:150]}...")

In [ ]:
# Tag den mest positive spam besked
most_pos_spam = df_sample[df_sample['label'] == 'spam'].nlargest(1, 'compound').iloc[0]
print(f"\nMest positive SPAM besked:")
print(f"Score: {most_pos_spam['compound']:.3f}")
print(f"Besked: {most_pos_spam['message']}")

doc = nlp(most_pos_spam['message'])
print("\nOrd-niveau analyse:")
for token in doc:
    if token._.polarity and token._.polarity != 0:
        print(f"  {token.text:20} - Polarity: {token._.polarity}")

display(HTML(asent.visualize(doc, style="prediction")))